# <div style="text-align: center; background-color: #00FFFF	; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">📊 EDA |Life Expectancy | Visualization</div>

<a id="1"></a>
# <div style="text-align: center; background-color: #838B8B; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">1. Import Necessary Libraries</div>

In [ ]:
!pip install altair
!pip install folium
!pip install pycountry-convert
!pip install geopy

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import missingno as mno
import plotly.offline as pyo 
import plotly.figure_factory as ff
import plotly.io as pio
from wordcloud import WordCloud
color_pal = sns.color_palette()
plt.style.use('seaborn-dark-palette')
plt.style.use('dark_background')

import nltk
import altair as alt
import re
from altair import datum
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
from geopy.geocoders import Nominatim

import warnings
warnings.filterwarnings('ignore')
sns.set_theme(style='darkgrid', palette='colorblind')
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()

#Model
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('/kaggle/input/life-expectancy-at-birth-across-the-globe/Life Expectancy at Birth.csv')

<a id="1"></a>
# <div style="text-align: center; background-color: #CDC8B1; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">2. Exploratory Data Analysis 📊 </div>

In [ ]:
df.head()

In [ ]:
col_names = df.columns.to_list()
df.rename(columns={ x:str(re.search( r'\((\d+)\)',x).group(1)) for x in col_names[7:]} , inplace=True)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.describe(include='object').T


In [ ]:
unique_values = df.nunique()
typpe_values = df.dtypes

summary_df = pd.DataFrame({'Unique Values': unique_values, 'Data Type': typpe_values})
print(summary_df)

# <div style="text-align: center; background-color: #CDC8B1; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">3. Null values</div>

In [ ]:
df.isna().sum()

In [ ]:
plt.figure(figsize = (30,10))
sns.barplot(df.isna())
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Fill missing values in categorical columns with 'Unknown'
categorical_columns = [ 'Human Development Groups','UNDP Developing Regions','HDI Rank (2021)']
df[categorical_columns] = df[categorical_columns].fillna('Unknown')


# <div style="text-align: center; background-color: #CDC8B1; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">4. Duplicate rows</div>

In [ ]:
# Finding duplicate rows
duplicate_rows = df[df.duplicated(keep='first')]

# Number of duplicate rows
num_duplicates = duplicate_rows.shape[0]

# Displaying the duplicate rows
print(f"Number of duplicate rows: {num_duplicates}")
duplicate_rows

# <div style="text-align: center; background-color: #CDC8B1; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">5. Data visualisation</div>

In [ ]:
# Import the ProfileReport class from the ydata_profiling library
from ydata_profiling import ProfileReport

# Create a comprehensive profile report for the DataFrame 'df'
# This report will contain various statistics, insights, and visualizations about the data
profile = ProfileReport(df)

In [ ]:
#profile                     # or this one
profile.to_notebook_iframe() # use this line to show the output

In [ ]:
# Columns for which you want to create plots
columns_to_plot = ['Continent', 'UNDP Developing Regions', 'Hemisphere']

for column in columns_to_plot:
    top10_values = df[column].value_counts()[:10]
    
    # Create a bar plot using Plotly Express
    fig = px.bar(
        x=top10_values.index,  # Use index as x values
        y=top10_values.values,
        color=top10_values.index,  # Use index for color variation
        labels={'x': column, 'y': 'Count'},
        title=f'Top 10 {column} Distribution',
        template='plotly_white'
    )
    fig.update_layout(
        font=dict(size=17, family="Franklin Gothic")
    )
    
    # Display the plot
    fig.show()

In [ ]:
# Assuming df is your DataFrame

# Convert non-numeric values to NaN
df["HDI Rank (2021)"] = pd.to_numeric(df["HDI Rank (2021)"], errors="coerce")

# Group by Continent and calculate the mean
grouped_df = df.groupby("Continent")["HDI Rank (2021)"].mean().reset_index()

In [ ]:

# Columns and y-labels for which you want to create plots
columns_to_plot = [
    ('Country', 'HDI Rank (2021)', 'HDI Rank (2021) by Country'),
    ('Continent', 'HDI Rank (2021)', 'HDI Rank (2021) by Continent'),
    ('Human Development Groups', 'HDI Rank (2021)', 'HDI Rank (2021) by Human Development Groups'),
    ('Country', '2021', ' Life Expectancy 2021 by Country'),
    ('Continent', '2021', ' Life Expectancy 2021 by Continent'),
    ('Human Development Groups','2021',' Life Expectancy 2021 by Human Development Groups'),
    ('Country', '2020', ' Life Expectancy 2020 by Country'),
    ('Continent', '2020', ' Life Expectancy 2020 by Continent'),
    ('Human Development Groups','2020',' Life Expectancy 2020 by Human Development Groups'),
    ('Country', '2019', ' Life Expectancy 2019 by Country'),
    ('Continent', '2019', ' Life Expectancy 2019 by Continent'),
    ('Human Development Groups','2019',' Life Expectancy 2019 by Human Development Groups'),
    ('Country', '2018', ' Life Expectancy 2018 by Country'),
    ('Continent', '2018', ' Life Expectancy 2018 by Continent'),
    ('Human Development Groups','2018',' Life Expectancy 2018 by Human Development Groups'),
    ('Country', '2017', ' Life Expectancy 2017 by Country'),
    ('Continent', '2017', ' Life Expectancy 2017 by Continent'),
    ('Human Development Groups','2017',' Life Expectancy 2017 by Human Development Groups'),
    ('Country', '2016', ' Life Expectancy 2016 by Country'),
    ('Continent', '2016', ' Life Expectancy 2016 by Continent'),
    ('Human Development Groups','2016',' Life Expectancy 2016 by Human Development Groups')

]

for column, y_label, title in columns_to_plot:
    data = df.groupby(column)[y_label].sum().reset_index()
    
    # Create a line plot using Plotly Express
    fig = px.line(
        data,
        x=column,
        y=y_label,
        title=title,
        height=800,
        line_shape='linear',  # Change the line shape (linear, spline, hv, vh, etc.)
        color_discrete_sequence=px.colors.qualitative.Pastel1  # Change the color palette
    )
    fig.update_layout(
        xaxis_title=column,
        yaxis_title=y_label,
        font=dict(size=17, family="Franklin Gothic"),
        template='plotly_white'  # Change the background template (plotly, ggplot2, seaborn, etc.)
    )
    
    # Display the plot
    fig.show()

In [ ]:
# Calculate the average HDI Rank for each country and sort in descending order
average_hdi_rank = df.groupby('Country')['HDI Rank (2021)'].mean().reset_index()
average_hdi_rank = average_hdi_rank.sort_values(by='HDI Rank (2021)', ascending=False)

# Select the top 10 countries with the highest average HDI Rank
top_10_high_hdi = average_hdi_rank.head(10)

# Create a new figure using Plotly
fig = go.Figure()

# Add a bar trace to the figure
fig.add_trace(go.Bar(
    x=top_10_high_hdi['Country'],  # x-values: countries
    y=top_10_high_hdi['HDI Rank (2021)']  # y-values: average HDI Ranks
))

# Update the layout and appearance of the plot
fig.update_layout(
    title='Top 10 Countries by Average HDI Rank',  # Set the title of the plot
    xaxis_title='Country',  # Label for the x-axis
    yaxis_title='Average HDI Rank',  # Label for the y-axis
    template='plotly_white',  # Use a white template for the plot
    font=dict(color='black'),  # Set font color to black
    height=650  # Set the height of the plot
)

# Initialize Plotly for notebook integration
pyo.init_notebook_mode(connected=True)

# Display the plot
pyo.iplot(fig)

In [ ]:
# Create a new DataFrame with columns for each year from 1990 to 2021
years = [str(year) for year in range(1990, 2022)]
yearly_life_expectancy = df[["Country"] + years]

# Calculate the average Life Expectancy for each year
yearly_life_expectancy["Average Life Expectancy"] = yearly_life_expectancy[years].mean(axis=1)

# Create a bar plot using Plotly Express
fig = px.bar(
    yearly_life_expectancy,
    x="Country",
    y="Average Life Expectancy",
    title="Average Life Expectancy by Year (1990-2021)",
    labels={"Average Life Expectancy": "Average Life Expectancy", "Country": "Country"},
    hover_data={year: ":.2f" for year in years},height=800

)

# Customize the layout if needed
fig.update_layout(xaxis_title="Country", yaxis_title="Average Life Expectancy")

# Show the plot
fig.show()


In [ ]:
# Create a new DataFrame with columns for each year from 1990 to 2021
years = [str(year) for year in range(1990, 2022)]
yearly_life_expectancy = df[["Continent"] + years]

# Calculate the average Life Expectancy for each year
yearly_life_expectancy["Average Life Expectancy"] = yearly_life_expectancy[years].mean(axis=1)

# Create a bar plot using Plotly Express
fig = px.bar(
    yearly_life_expectancy,
    x="Continent",
    y="Average Life Expectancy",
    title="Average Life Expectancy by Year (1990-2021)",
    labels={"Average Life Expectancy": "Average Life Expectancy", "Continent": "Continent"},
    hover_data={year: ":.2f" for year in years},height=800

)

# Customize the layout if needed
fig.update_layout(xaxis_title="Continent", yaxis_title="Average Life Expectancy")

# Show the plot
fig.show()


In [ ]:
# Columns and titles for which you want to create sunburst charts
columns_to_plot = [
    ('Continent', 'Distribution of Continent'),
    ('Hemisphere','Distribution of Human Hemisphere')
]

for column, title in columns_to_plot:
    # Create a sunburst chart using Plotly Express
    fig = px.sunburst(df, path=[column], color_discrete_sequence=px.colors.qualitative.Set1
                     )
    fig.update_layout(title_text=title, height=500)
    
    # Display the plot
    fig.show()

In [ ]:
# Drop rows with missing values in the specified columns
df_cleaned = df.dropna(subset=["HDI Rank (2021)", "Human Development Groups"])

# Create a scatter plot using Plotly Express
fig = px.scatter(
    df_cleaned,
    x="HDI Rank (2021)",
    y="Human Development Groups",
    size="HDI Rank (2021)",
    color="Continent",
    hover_name="Country",
    log_x=True,
    size_max=60
)
fig.show()

In [ ]:
def Freq_df(cleanwordlist):
    Freq_dist_nltk = nltk.FreqDist(cleanwordlist)
    df_freq = pd.DataFrame.from_dict(Freq_dist_nltk, orient='index')
    df_freq.columns = ['Frequency']
    df_freq.index.name = 'Term'
    df_freq = df_freq.sort_values(by=['Frequency'],ascending=False)
    df_freq = df_freq.reset_index()
    return df_freq

def Word_Cloud(data, color_background, colormap, title):
    plt.figure(figsize = (20,15))
    wc = WordCloud(width=1200, 
               height=600, 
               max_words=50,
               colormap= colormap,
               max_font_size = 100,
               random_state=88, 
               background_color=color_background).generate_from_frequencies(data)
    plt.imshow(wc, interpolation='bilinear')
    plt.title(title, fontsize=20)
    plt.axis('off')
    plt.show()

In [ ]:
freq_df = Freq_df(df['Country'].values.tolist())
data = dict(zip(freq_df['Term'].tolist(), freq_df['Frequency'].tolist()))
data = freq_df.set_index('Term').to_dict()['Frequency']

Word_Cloud(data ,'black','RdBu', 'WordCloud of Country')

In [ ]:
# Melt the dataframe to make it suitable for animation
melted_df = df.melt(id_vars=['ISO3', 'Country', 'Continent'],
                    value_vars=[str(year) for year in range(1990, 2022)],
                    var_name='Year', value_name='Life Expectancy')

# Create an animated map visualization
fig = px.choropleth(
    melted_df,
    locations="ISO3",
    color="Life Expectancy",
    hover_name="Country",
    hover_data=["Continent", "Year"],
    animation_frame="Year",
    title="Life Expectancy at Birth by Continent (1990-2021)",
    color_continuous_scale="viridis",
    range_color=(40, 90),  # Adjust the color scale range
    projection="natural earth"
)

# Update layout
fig.update_layout(geo=dict(showcoastlines=True))

# Show the animated map
fig.show()

In [ ]:
fig1 = px.choropleth(df,
                     locations='Country',
                     locationmode='country names',
                     color='2021',
                     color_continuous_scale=px.colors.sequential.Viridis,
                     template='plotly_dark',
                     title = '2021 Life Expectancy At Birth')

fig1.update_layout(font = dict(size = 17, family="Franklin Gothic"))

fig1.show()

In [ ]:
fig2 = px.choropleth(df,
                     locations='Country',
                     locationmode='country names',
                     color='Human Development Groups',
                     template='plotly_white',
                     title = 'Human Development Groups')

fig2.update_layout(font = dict(size = 17, family="Franklin Gothic"))

fig2.show()


# <div style="text-align: center; background-color: #CDC8B1; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">6. Categorical
</div>


In [ ]:
# Select columns with object (categorical) data types
num_cols = df.select_dtypes(include='object').columns.tolist()

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Apply Label Encoding to the selected numerical columns
for x in num_cols:  
    df[x] = label_encoder.fit_transform(df[x])

# Now, your categorical columns (excluding column 0) have been converted to numerical values
df.head()


# <div style="text-align: center; background-color: #CDC8B1; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">7. Reduce dimensionality


</div>


In [ ]:
# Calculate the correlation matrix
correlation_matrix = df.corr()

# Create a correlation heatmap using Plotly Express
fig = px.imshow(
    correlation_matrix,  # Matrix containing the data
    labels=dict(x="Features", y="Features", color="Correlation"),  # Customize labels
    x=correlation_matrix.columns,  # x-values: Features
    y=correlation_matrix.columns,  # y-values: Features
    color_continuous_scale='blues',  # Set the color scale
    title='Correlation Heatmap',  # Set the title of the plot
    height=1000  # Set the height of the plot
)

# Display the plot
fig.show()

In [ ]:
print('Top 5 Most Positively Correlated to the To Human Development Groups')
correlation_matrix['Human Development Groups'].sort_values(ascending=False).head(5)

In [ ]:
print('Top 5 Most Positively Correlated to Human Development Groups ')
correlation_matrix['Human Development Groups'].sort_values(ascending=True).head(5)

In [ ]:
columns_to_drop = [col for col in correlation_matrix.columns if abs(correlation_matrix.loc['Human Development Groups', col]) < 0.3]
print('number of columns to drop ' ,len(columns_to_drop))

In [ ]:
df = df.drop(columns_to_drop, axis=1)
df.shape

In [ ]:
df = df.dropna()


# <div style="text-align: center; background-color: #CDC8B1; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">8. spliting the dataset

</div>


In [ ]:
X = df.drop(columns=["Human Development Groups"])
y = df['Human Development Groups']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)  # Using stratified sampling
# Display the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


# <div style="text-align: center; background-color: #CDC8B1; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">9. Model Building and Analysis




</div>

In [ ]:
# Encode the target variable
label_encoder = LabelEncoder()
df['Human Development Groups'] = label_encoder.fit_transform(df['Human Development Groups'])

In [ ]:
# Initialize a Random Forest Classifier
clf = RandomForestClassifier(random_state=42)

# Train the model
clf.fit(X_train, y_train)

# Predict using the trained model
y_pred = clf.predict(X_test)

In [ ]:
# Evaluate the model
report = classification_report(y_test, y_pred)

# Split the classification report into lines
report_lines = report.split('\n')

# Create header row
header_row = report_lines[0].split()

# Create rows for each class
table_rows = [row.split() for row in report_lines[2:-5]]

# Create the table using Plotly
table = go.Figure(data=[go.Table(header=dict(values=header_row),
                 cells=dict(values=list(zip(*table_rows))))
                 ])

# Show the table
table.show()


# <div style="text-align: center; background-color: #CDC8B1; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">10. feature importances




</div>

In [ ]:
importances = clf.feature_importances_
feature_names = X.columns
feature_importance_dict = dict(zip(feature_names, importances))
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

top_n = 5  # Set the number of top features to display
top_feature_names, top_importances = zip(*sorted_feature_importance[:top_n])

fig = px.bar(
    x=top_importances,
    y=top_feature_names,
    orientation='h',
    title='Top 5 Feature Importance',
    labels={'x': 'Importance', 'y': 'Feature'},
    color=top_importances,  # Color bars by importance values
    color_continuous_scale='reds',  # Choose a color scale
)

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')

fig.show()
